## This notebook shows all the means to create a Diagram and set the spots
* To instanciate a `Diagram` allows you to apply efficiently filters based on image processing.
* They are several means to instanciate a `Diagram`, some of them are betters than overs.

In [ ]:
%matplotlib inline

import itertools
import random

import matplotlib.pyplot as plt
import numpy as np
import torch

from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram  # you don't need to import more to access all the filters

# you can juste import the main module
# import laueimproc  # then acces to `Diagram` with laueimproc.Diagram

### Firstly, we have to create an "empty" diagram from an image
* A diagram is an abstraction of a Laue diffraction image, containing some spots.
* Before having spots, it is firstaval an image!

In [ ]:
print("*** How to init an empty diagram? ***")
print(Diagram.__init__.__doc__)
print("***   What are the attributes ?   ***")
for cls in Diagram.mro()[1::-1]:
    print(cls.__doc__)

In [ ]:
print(file := next(get_samples().iterdir()))  # we get the path of a Laue diffraction image on the disk
diag = Diagram(file)  # it is the cannonical way to instanciate a Diagram
print()
print(diag)

# try not to do that because it is very dangerous!
arr_numpy = np.array(diag.image)  # let consider it comes from somewhere else
arr_torch = torch.from_numpy(arr_numpy)
diag_from_numpy = Diagram(arr_numpy)
diag_from_torch = Diagram(arr_torch)

## Secondly, the spots must be initialized
* The idea is to init the diagram with a lot of spots, then to throw the spots you don't want by filtering.
* It is recommend initializing spots before accessing the `.spot` attribute. If you don't, a default peaks serach will be applied and you will get a warning.

In [ ]:
# 1: use the internal peaks_search, it removes the background in the rois
diag_1 = diag.clone()  # real deep copy
diag_1.find_spots(threshold=3.5)  # parameters are optional, there are other, see the doc
print(diag_1)
diag_1.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()  # display to see the bboxes

# 2: use external spots of type Spot
new_spots = diag_1.spots[::5]  # let imagine it comes from somewhere else
diag_2 = diag.clone()  # new empty the diagram
diag_2.spots = new_spots  # set the spots
print(diag_2)
diag_2.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

# 3: use external bounding boxes
bboxes = [  # can be an array, a tensor or any iterators as well
    (i, j, random.randint(5, 25), random.randint(5, 25))  # numpy convention (*anchor, *shape)
    for i, j in itertools.product(range(0, min(diag.image.shape)-25, 100), repeat=2)
]
diag_3 = diag.clone()  # new empty the diagram
diag_3.spots = bboxes  # set the spots
print(diag_3)
diag_3.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

# 4: use external anchors and rois
anchors = list(itertools.product(range(0, min(diag.image.shape)-25, 100), repeat=2))  # numpy convention
rois = [np.empty((random.randint(5, 25), random.randint(5, 25))) for _ in anchors]  # roi patches
diag_4 = diag.clone()  # new empty the diagram
diag_4.spots = anchors, rois
print(diag_4)
diag_4.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

## Attribute values before and after spot initialization
* Some attributes are set to != None when spots are initialized.

In [ ]:
# 0: on uninitialized diagram
print("spots of diag:", diag.spots)
print("bboxes of diag:", diag.bboxes)
print("rois of diag:", diag.rois)

# 1: on any initialized diagram
initialized_diag = diag_2  # you can change here for testing
print("spots:", initialized_diag.spots)
print("bboxes:", initialized_diag.bboxes)
print("rois:", initialized_diag.rois)